In [6]:
import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import joblib
from datetime import date 
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Stock list for 20 companies
stocks = ["AAPL","MSFT","GOOGL","AMZN","META","TSLA","NFLX","NVDA","INTC","AMD","BABA",
          "ORCL","PYPL","ADBE","CRM","IBM","KO","PEP","DIS","UBER"]

start = "2015-01-01"
end   = date.today().strftime("%Y-%m-%d")
print("Downloading stock data...")

# Download data for all stocks
data = yf.download(tickers=stocks, start=start, end=end)
data = data[['Open', 'High', 'Low', 'Close']]

# Dictionaries to store results
models = {}      
scalers = {}      
histories = {}  
predictions = {}  

# Function to create sequences
def create_sequences(dataset, lookback=60):
    x, y = [], []
    for i in range(lookback, len(dataset)):
        x.append(dataset[i-lookback:i, :])   # all features (OHLC + SMA30)
        y.append(dataset[i, 3])              # 3rd index = Close
    return np.array(x), np.array(y)

for ticker in stocks:
    print(f"Processing {ticker}...")
    
    try:
        # Extract OHLC for one ticker - CORRECTED
        df_ticker = data.xs(ticker, level=1, axis=1)
        df_ticker = df_ticker[['Open', 'High', 'Low', 'Close']].fillna(method="ffill")
        
        # Add SMA30
        df_ticker["SMA30"] = df_ticker["Close"].rolling(window=30).mean().fillna(method="bfill")
        
        # Create a new scaler for EACH ticker
        scaler = MinMaxScaler(feature_range=(0,1))
        scaled_data = scaler.fit_transform(df_ticker)
        
        # Save scaler for this ticker
        scalers[ticker] = scaler
        joblib.dump(scaler, f"{ticker}_scaler.save")
        
        # Sequence creation
        x, y = create_sequences(scaled_data, 60)
        
        # Train-test split
        train_size = int(len(x) * 0.8)
        x_train, x_test = x[:train_size], x[train_size:]
        y_train, y_test = y[:train_size], y[train_size:]
        
        print(f"   Data shape for {ticker}: {x_train.shape}")
        
        # Build model
        model = Sequential()
        model.add(LSTM(50, return_sequences=True, input_shape=(x_train.shape[1], x_train.shape[2])))
        model.add(Dropout(0.2))
        model.add(LSTM(60, return_sequences=True))
        model.add(Dropout(0.3))
        model.add(LSTM(80, return_sequences=True))
        model.add(Dropout(0.4))
        model.add(LSTM(120))
        model.add(Dropout(0.5))
        model.add(Dense(1))
        
        model.compile(optimizer='adam', loss='mean_squared_error')
        
        # Callbacks
        early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
        checkpoint = ModelCheckpoint(f"{ticker}_best_lstm_model.h5", monitor='val_loss',
                                     save_best_only=True, verbose=1)
        
        # Train model
        print(f"   Training {ticker} model...")
        history = model.fit(
            x_train, y_train,
            epochs=100,  # Reduced for testing, you can increase later
            batch_size=32,
            validation_data=(x_test, y_test),
            callbacks=[early_stop, checkpoint],
            verbose=1
        )
        
        # Save model and history
        model.save(f"{ticker}_lstm_model.h5")
        models[ticker] = model
        histories[ticker] = history
        
        print(f"Completed {ticker}")
        
    except Exception as e:
        print(f"Error processing {ticker}: {str(e)}")
        continue

print("Training completed!")

C:\Users\user\AppData\Local\Temp\ipykernel_8208\120555688.py:23: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(tickers=stocks, start=start, end=end)
[*****                 10%                       ]  2 of 20 completed

[*********************100%***********************]  20 of 20 completed
C:\Users\user\AppData\Local\Temp\ipykernel_8208\120555688.py:46: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_ticker = df_ticker[['Open', 'High', 'Low', 'Close']].fillna(method="ffill")
C:\Users\user\AppData\Local\Temp\ipykernel_8208\120555688.py:49: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_ticker["SMA30"] = df_ticker["Close"].rolling(window=30).mean().fillna(method="bfill")
c:\Users\user\OneDrive - City University\Desktop\Python B\.conda\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


🚀 Processing AAPL...
   Data shape for AAPL: (2096, 60, 5)
   Training AAPL model...
Epoch 1/100
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - loss: 0.0193
Epoch 1: val_loss improved from None to 0.02137, saving model to AAPL_best_lstm_model.h5


66/66 ━━━━━━━━━━━━━━━━━━━━ 9s 70ms/step - loss: 0.0087 - val_loss: 0.0214
Epoch 2/100
65/66 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - loss: 0.0028
Epoch 2: val_loss improved from 0.02137 to 0.00289, saving model to AAPL_best_lstm_model.h5


66/66 ━━━━━━━━━━━━━━━━━━━━ 4s 64ms/step - loss: 0.0024 - val_loss: 0.0029
Epoch 3/100
65/66 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - loss: 0.0021
Epoch 3: val_loss did not improve from 0.00289
66/66 ━━━━━━━━━━━━━━━━━━━━ 4s 64ms/step - loss: 0.0025 - val_loss: 0.0032
Epoch 4/100
65/66 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - loss: 0.0022
Epoch 4: val_loss improved from 0.00289 to 0.00236, saving model to AAPL_best_lstm_model.h5


...

✅ Completed DIS
🚀 Processing UBER...
   Data shape for UBER: (2096, 60, 5)
   Training UBER model...
Epoch 1/100


C:\Users\user\AppData\Local\Temp\ipykernel_8208\120555688.py:46: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_ticker = df_ticker[['Open', 'High', 'Low', 'Close']].fillna(method="ffill")
C:\Users\user\AppData\Local\Temp\ipykernel_8208\120555688.py:49: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_ticker["SMA30"] = df_ticker["Close"].rolling(window=30).mean().fillna(method="bfill")
c:\Users\user\OneDrive - City University\Desktop\Python B\.conda\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step - loss: nan
Epoch 1: val_loss improved from None to nan, saving model to UBER_best_lstm_model.h5


66/66 ━━━━━━━━━━━━━━━━━━━━ 10s 90ms/step - loss: nan - val_loss: nan
Epoch 2/100
65/66 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step - loss: nan
Epoch 2: val_loss did not improve from nan
66/66 ━━━━━━━━━━━━━━━━━━━━ 5s 81ms/step - loss: nan - val_loss: nan
Epoch 3/100
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step - loss: nan
Epoch 3: val_loss did not improve from nan
66/66 ━━━━━━━━━━━━━━━━━━━━ 5s 82ms/step - loss: nan - val_loss: nan
Epoch 4/100
65/66 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step - loss: nan
Epoch 4: val_loss did not improve from nan
66/66 ━━━━━━━━━━━━━━━━━━━━ 5s 82ms/step - loss: nan - val_loss: nan
Epoch 5/100
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step - loss: nan
Epoch 5: val_loss did not improve from nan
66/66 ━━━━━━━━━━━━━━━━━━━━ 5s 83ms/step - loss: nan - val_loss: nan
Epoch 6/100
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step - loss: nan
Epoch 6: val_loss did not improve from nan
66/66 ━━━━━━━━━━━━━━━━━━━━ 5s 82ms/step - loss: nan - val_loss: nan
Epoch 7/100
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step - lo

✅ Completed UBER
Training completed!


In [8]:
import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import joblib
from datetime import date, timedelta
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import load_model

# Function to create sequences (same as training)
def create_sequences(dataset, lookback=60):
    x, y = [], []
    for i in range(lookback, len(dataset)):
        x.append(dataset[i-lookback:i, :])   # all features (OHLC + SMA30)
        y.append(dataset[i, 3])              # 3rd index = Close
    return np.array(x), np.array(y)

# Function to test a single model
def test_model(ticker, start_date="2023-01-01", end_date=None):
    if end_date is None:
        end_date = date.today().strftime("%Y-%m-%d")
    
    print(f"Testing {ticker}...")
    
    try:
        # Load the saved model and scaler
        model = load_model(f"{ticker}_lstm_model.h5")
        scaler = joblib.load(f"{ticker}_scaler.save")
        
        # Download fresh data for testing period
        test_data = yf.download(ticker, start=start_date, end=end_date)
        test_data = test_data[['Open', 'High', 'Low', 'Close']]
        
        # Add SMA30
        test_data["SMA30"] = test_data["Close"].rolling(window=30).mean().fillna(method="bfill")
        
        # Scale the test data using the same scaler
        scaled_test_data = scaler.transform(test_data)
        
        # Create sequences for testing
        x_test, y_test = create_sequences(scaled_test_data, 60)
        
        if len(x_test) == 0:
            print(f"Not enough data to test {ticker}")
            return None, None, None
        
        # Make predictions
        predictions = model.predict(x_test)
        
        # Inverse transform predictions and actual values
        # Create dummy arrays for inverse transformation
        dummy_array_pred = np.zeros((len(predictions), scaled_test_data.shape[1]))
        dummy_array_pred[:, 3] = predictions.flatten()  # Close price is at index 3
        predictions_actual = scaler.inverse_transform(dummy_array_pred)[:, 3]
        
        dummy_array_actual = np.zeros((len(y_test), scaled_test_data.shape[1]))
        dummy_array_actual[:, 3] = y_test
        y_test_actual = scaler.inverse_transform(dummy_array_actual)[:, 3]
        
        # Calculate metrics
        mse = mean_squared_error(y_test_actual, predictions_actual)
        mae = mean_absolute_error(y_test_actual, predictions_actual)
        rmse = np.sqrt(mse)
        r2 = r2_score(y_test_actual, predictions_actual)
        
        # Calculate accuracy (percentage of correct direction predictions)
        direction_accuracy = np.mean(
            (np.sign(predictions_actual[1:] - predictions_actual[:-1]) == 
             np.sign(y_test_actual[1:] - y_test_actual[:-1])).astype(int)
        ) * 100
        
        print(f" {ticker} Test Results:")
        print(f"   MSE: {mse:.4f}, MAE: {mae:.4f}, RMSE: {rmse:.4f}")
        print(f"   R² Score: {r2:.4f}")
        print(f"   Direction Accuracy: {direction_accuracy:.2f}%")
        
        # Plot results
        plt.figure(figsize=(12, 6))
        plt.plot(test_data.index[60:60+len(y_test_actual)], y_test_actual, label='Actual', linewidth=2)
        plt.plot(test_data.index[60:60+len(predictions_actual)], predictions_actual, label='Predicted', linewidth=2)
        plt.title(f'{ticker} Stock Price Prediction')
        plt.xlabel('Date')
        plt.ylabel('Price ($)')
        plt.legend()
        plt.grid(True)
        plt.savefig(f"{ticker}_prediction_test.png")
        plt.close()
        
        # Save results to CSV
        results_df = pd.DataFrame({
            'Date': test_data.index[60:60+len(y_test_actual)],
            'Actual': y_test_actual,
            'Predicted': predictions_actual
        })
        results_df.to_csv(f"{ticker}_test_results.csv", index=False)
        
        return y_test_actual, predictions_actual, {
            'mse': mse, 'mae': mae, 'rmse': rmse, 'r2': r2, 'direction_accuracy': direction_accuracy
        }
        
    except Exception as e:
        print(f"Error testing {ticker}: {str(e)}")
        return None, None, None

# Function to test all models
def test_all_models(start_date="2023-01-01", end_date=None):
    results = {}
    
    for ticker in stocks:
        try:
            actual, predicted, metrics = test_model(ticker, start_date, end_date)
            if metrics:
                results[ticker] = metrics
        except:
            continue
    
    # Print summary of all results
    if results:
        print("\n" + "="*60)
        print("SUMMARY OF ALL MODEL PERFORMANCES")
        print("="*60)
        
        results_df = pd.DataFrame(results).T
        results_df = results_df.sort_values('rmse')
        
        print(results_df)
        
        # Plot comparison of RMSE values
        plt.figure(figsize=(12, 8))
        results_df['rmse'].sort_values().plot(kind='barh')
        plt.title('RMSE Comparison Across Stocks')
        plt.xlabel('RMSE')
        plt.tight_layout()
        plt.savefig("all_models_rmse_comparison.png")
        plt.close()
        
        # Save summary results
        results_df.to_csv("all_models_test_results.csv")
        
    return results

# Function to predict future prices
def predict_future(ticker, days=30):
    print(f"🔮 Predicting next {days} days for {ticker}...")
    
    try:
        # Load the saved model and scaler
        model = load_model(f"{ticker}_lstm_model.h5")
        scaler = joblib.load(f"{ticker}_scaler.save")
        
        # Download recent data (last 60+ days)
        end_date = date.today()
        start_date = end_date - timedelta(days=120)  # Get more data than needed
        recent_data = yf.download(ticker, start=start_date, end=end_date)
        recent_data = recent_data[['Open', 'High', 'Low', 'Close']]
        
        # Add SMA30
        recent_data["SMA30"] = recent_data["Close"].rolling(window=30).mean().fillna(method="bfill")
        
        # Get the last 60 days of data
        last_60_days = recent_data.iloc[-60:].copy()
        scaled_data = scaler.transform(last_60_days)
        
        future_predictions = []
        current_sequence = scaled_data.copy()
        
        for _ in range(days):
            # Reshape the sequence for prediction
            x = current_sequence[-60:].reshape(1, 60, scaled_data.shape[1])
            
            # Predict next day
            next_pred = model.predict(x, verbose=0)
            
            # Create a new row for the prediction
            new_row = current_sequence[-1].copy()  # Copy the last row
            new_row[3] = next_pred[0, 0]  # Update the Close price
            
            # For other features, we can use simple assumptions
            # In a real application, you might want to predict these too or use better methods
            new_row[0] = new_row[3] * 0.99  # Open slightly lower than close
            new_row[1] = new_row[3] * 1.01  # High slightly higher than close
            new_row[2] = new_row[3] * 0.98  # Low slightly lower than close
            
            # Append to sequence and future predictions
            current_sequence = np.vstack([current_sequence, new_row])
            future_predictions.append(new_row)
        
        # Inverse transform the future predictions
        future_predictions = np.array(future_predictions)
        future_predictions_actual = scaler.inverse_transform(future_predictions)[:, 3]  # Close prices
        
        # Create future dates
        last_date = recent_data.index[-1]
        future_dates = [last_date + timedelta(days=i) for i in range(1, days+1)]
        
        # Plot historical and future predictions
        plt.figure(figsize=(14, 7))
        plt.plot(recent_data.index[-60:], recent_data['Close'][-60:], label='Historical', linewidth=2)
        plt.plot(future_dates, future_predictions_actual, label='Predicted', linewidth=2, color='red')
        plt.title(f'{ticker} Future Price Prediction (Next {days} Days)')
        plt.xlabel('Date')
        plt.ylabel('Price ($)')
        plt.legend()
        plt.grid(True)
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.savefig(f"{ticker}_future_prediction.png")
        plt.close()
        
        # Create results DataFrame
        future_df = pd.DataFrame({
            'Date': future_dates,
            'Predicted_Close': future_predictions_actual
        })
        
        future_df.to_csv(f"{ticker}_future_predictions.csv", index=False)
        
        print(f"Future predictions saved for {ticker}")
        return future_df
        
    except Exception as e:
        print(f"Error predicting future for {ticker}: {str(e)}")
        return None

# Main execution for testing
print("\n" + "="*50)
print("STARTING MODEL TESTING")
print("="*50)

# Test all models
results = test_all_models()

# Predict future for a specific stock (e.g., AAPL)
predict_future("AAPL", days=30)

print("\nTesting completed! Check the generated charts and CSV files for results.")


STARTING MODEL TESTING
Testing AAPL...


C:\Users\user\AppData\Local\Temp\ipykernel_8208\621903705.py:32: FutureWarning: YF.download() has changed argument auto_adjust default to True
  test_data = yf.download(ticker, start=start_date, end=end_date)
[*********************100%***********************]  1 of 1 completed
C:\Users\user\AppData\Local\Temp\ipykernel_8208\621903705.py:36: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  test_data["SMA30"] = test_data["Close"].rolling(window=30).mean().fillna(method="bfill")
c:\Users\user\OneDrive - City University\Desktop\Python B\.conda\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 55ms/step
 AAPL Test Results:
   MSE: 49.5718, MAE: 5.4800, RMSE: 7.0407
   R² Score: 0.9172
   Direction Accuracy: 50.83%
Testing MSFT...


C:\Users\user\AppData\Local\Temp\ipykernel_8208\621903705.py:32: FutureWarning: YF.download() has changed argument auto_adjust default to True
  test_data = yf.download(ticker, start=start_date, end=end_date)
[*********************100%***********************]  1 of 1 completed
C:\Users\user\AppData\Local\Temp\ipykernel_8208\621903705.py:36: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  test_data["SMA30"] = test_data["Close"].rolling(window=30).mean().fillna(method="bfill")
c:\Users\user\OneDrive - City University\Desktop\Python B\.conda\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 47ms/step
 MSFT Test Results:
   MSE: 270.2692, MAE: 12.8824, RMSE: 16.4399
   R² Score: 0.9179
   Direction Accuracy: 48.68%
Testing GOOGL...


...

19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 44ms/step
 DIS Test Results:
   MSE: 3.7484, MAE: 1.3355, RMSE: 1.9361
   R² Score: 0.9746
   Direction Accuracy: 47.69%
Testing UBER...


C:\Users\user\AppData\Local\Temp\ipykernel_8208\621903705.py:32: FutureWarning: YF.download() has changed argument auto_adjust default to True
  test_data = yf.download(ticker, start=start_date, end=end_date)
[*********************100%***********************]  1 of 1 completed
C:\Users\user\AppData\Local\Temp\ipykernel_8208\621903705.py:36: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  test_data["SMA30"] = test_data["Close"].rolling(window=30).mean().fillna(method="bfill")
c:\Users\user\OneDrive - City University\Desktop\Python B\.conda\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 45ms/step
Error testing UBER: Input contains NaN.

SUMMARY OF ALL MODEL PERFORMANCES
               mse        mae       rmse        r2  direction_accuracy
INTC      1.156710   0.766152   1.075504  0.983457           51.815182
KO        2.317023   1.176210   1.522177  0.933602           51.650165
DIS       3.748364   1.335531   1.936069  0.974583           47.689769
BABA     10.912262   2.400726   3.303371  0.970235           48.184818
PEP      13.939213   3.033120   3.733526  0.917584           48.184818
IBM      21.689557   3.177706   4.657205  0.991187           54.620462
AMD      31.206680   4.251078   5.586294  0.962133           50.825083
NVDA     44.550501   4.967310   6.674616  0.977393           50.000000
AAPL     49.571753   5.480042   7.040721  0.917244           50.825083
ORCL     55.430621   5.083592   7.445174  0.964686           51.980198
AMZN     59.886449   6.025656   7.738634  0.955267           48.844884
GOOGL    65.740118   6.494938   8

C:\Users\user\AppData\Local\Temp\ipykernel_8208\621903705.py:155: FutureWarning: YF.download() has changed argument auto_adjust default to True
  recent_data = yf.download(ticker, start=start_date, end=end_date)
[*********************100%***********************]  1 of 1 completed
C:\Users\user\AppData\Local\Temp\ipykernel_8208\621903705.py:159: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  recent_data["SMA30"] = recent_data["Close"].rolling(window=30).mean().fillna(method="bfill")
c:\Users\user\OneDrive - City University\Desktop\Python B\.conda\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


Future predictions saved for AAPL

Testing completed! Check the generated charts and CSV files for results.
